## Introdução

Aqui irei realizar a etapa de Feature Engineering na base de dados `bureau`.

Esta etapa está dividida em kernels diferentes devido o consumo de memória para carregar e manipular as bases.

Os procedimentos realizados nas outras bases podem ser encontrados nos arquivos `02_feat_eng_<nome-da-base>.ipynb`

O objetivo desta etapa consiste, principalmente, em criar variáveis (`book de variáveis`). Ao criar novas variáveis com base nas variáveis existentes, é possível capturar informações adicionais que podem não estar explicitamente presentes nos dados originais.

Devido o volume de dados, optei por utilizar o PySpark em conjunto com o SparkSQL para as operações a seguir.

* Sobre os Dados

A base `bureau` possui dados de crédito de outras instituições financeiras.

Segundo os Metadados disponibilizados, essas são as informações contidas aqui:

``SK_BUREAU_ID``: ID recodificado do crédito do Bureau de Crédito (codificação única para cada aplicação) - Será usado para trazer os dados da tabela `bureau balance`.

``SK_ID_CURR``: ID do empréstimo em nossa amostra - um empréstimo em nossa amostra pode ter 0, 1, 2 ou mais créditos anteriores relacionados no bureau de crédito.

`CREDIT_ACTIVE`: Status dos créditos reportados pelo Bureau de Crédito (CB).

`CREDIT_CURRENCY`: Moeda recodificada do crédito do Bureau de Crédito.

`DAYS_CREDIT`: Quantos dias antes da aplicação atual o cliente solicitou crédito ao Bureau de Crédito.

`CREDIT_DAY_OVERDUE`: Número de dias em atraso no crédito do CB no momento da aplicação para o empréstimo relacionado em nossa amostra.

`DAYS_CREDIT_ENDDATE`: Duração restante do crédito do CB (em dias) no momento da aplicação no Home Credit.

`DAYS_ENDDATE_FACT`: Dias desde que o crédito do CB foi encerrado no momento da aplicação no Home Credit (apenas para créditos encerrados).

`AMT_CREDIT_MAX_OVERDUE`: Valor máximo em atraso no crédito do Bureau de Crédito até o momento (na data de aplicação do empréstimo em nossa amostra).

`CNT_CREDIT_PROLONG`: Quantas vezes o crédito do Bureau de Crédito foi prolongado.

`AMT_CREDIT_SUM`: Valor atual do crédito para o crédito do Bureau de Crédito.

`AMT_CREDIT_SUM_DEBT`: Dívida atual no crédito do Bureau de Crédito.

`AMT_CREDIT_SUM_LIMIT`: Limite de crédito atual do cartão de crédito relatado no Bureau de Crédito.

`AMT_CREDIT_SUM_OVERDUE`: Valor atual em atraso no crédito do Bureau de Crédito.

`CREDIT_TYPE`: Tipo de crédito do Bureau de Crédito (Carro, dinheiro, ...).

`DAYS_CREDIT_UPDATE`: Quantos dias antes da aplicação do empréstimo foi recebida a última informação sobre o crédito do Bureau de Crédito.

`AMT_ANNUITY`: Anuidade do crédito do Bureau de Crédito.

## Utils

* Importando as bibliotecas que irei utilizar

In [14]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,min, max, sum, round, col, median, count
spark = SparkSession.builder.appName("FeatureEng").getOrCreate()
from warnings import filterwarnings
filterwarnings('ignore')

ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente

## Feature Engineering - Previous Application

In [2]:
prev_app = spark.read.csv('./DATASETS/previous_application.csv', inferSchema= True, header= True)
prev_app.count()

1670214

In [3]:
prev_app.show(n=5)

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+--------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+--------------------+----------------+--------------------+-----------+----------------+--------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURPOSE|NAME

* Checando a quantidade de linhas da tabela final (para validação posterior)

In [3]:
prev_app.groupBy("SK_ID_PREV").count().count()

1670214

* Verificando o Schema

In [5]:
prev_app.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_APPLICATION: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_DOWN_PAYMENT: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- WEEKDAY_APPR_PROCESS_START: string (nullable = true)
 |-- HOUR_APPR_PROCESS_START: integer (nullable = true)
 |-- FLAG_LAST_APPL_PER_CONTRACT: string (nullable = true)
 |-- NFLAG_LAST_APPL_IN_DAY: integer (nullable = true)
 |-- RATE_DOWN_PAYMENT: double (nullable = true)
 |-- RATE_INTEREST_PRIMARY: double (nullable = true)
 |-- RATE_INTEREST_PRIVILEGED: double (nullable = true)
 |-- NAME_CASH_LOAN_PURPOSE: string (nullable = true)
 |-- NAME_CONTRACT_STATUS: string (nullable = true)
 |-- DAYS_DECISION: integer (nullable = true)
 |-- NAME_PAYMENT_TYPE: string (nullable = true)
 |-- CODE_REJECT_REASON: string (nullable = t

### 2. Criação de Flags Temporais

In [3]:
# Criando uma View
prev_app.createOrReplaceTempView('prev_app')

temp01 = spark.sql("""
SELECT
    *,
        CASE
            WHEN DAYS_DECISION >= -90 THEN 1
        ELSE 0
    END AS FL_U3M,
        CASE
            WHEN DAYS_DECISION >= -180 THEN 1
        ELSE 0
    END AS FL_U6M,
        CASE
            WHEN DAYS_DECISION >= -270 THEN 1
        ELSE 0
    END AS FL_U9M,
        CASE
            WHEN DAYS_DECISION >= -360 THEN 1
        ELSE 0
    END AS FL_U12M
FROM
    prev_app
ORDER BY
    `SK_ID_PREV`;
""")

temp01.createOrReplaceTempView('temp01')
temp01.count()

1670214

### 3. Realizando os Joins com as Bases Tratadas

* Carregando a POS Cash Balance, Credit Card Balance, Instalments Payments

In [4]:
pos_cash = spark.read.parquet("./BASES_FEAT_ENG/POS_CASH_BALANCE_FEAT_ENG")
instalments = spark.read.parquet("./BASES_FEAT_ENG/INSTALMENTS_PAYMENTS_FEAT_ENG")
credit_balance = spark.read.parquet("./BASES_FEAT_ENG/CREDIT_CARD_BALANCE_FEAT_ENG")

* Realizando os Joins utilizando a **SK_ID_PREV** como chave

In [5]:
temp02 = temp01.join(pos_cash, "SK_ID_PREV", how = 'left').join(credit_balance, "SK_ID_PREV", how = 'left').join(instalments, "SK_ID_PREV", how = 'left')
temp02.count()

1670214

In [6]:
len(temp02.columns)

567

### 4. Criação das Variáveis (Agrupadas)

* Filtrando as colunas que não serão agregadas (Optei por remover devido limitações computacionais)

In [7]:
flags_temporais = ['FL_U3M', 'FL_U6M', 'FL_U9M','FL_U12M']

In [8]:
# Removendo as colunas de IDs
agg_cols = [col for col in temp02.columns if "SK_ID" not in col]
len(agg_cols)

565

In [9]:
# Removendo as colunas de Flags Temporais
for _ in flags_temporais:
    agg_cols.remove(_)
len(agg_cols)

561

In [10]:
# Filtrando as colunas categóricas (não irão entrar nas agregações)
catcols = [cat_col[0] for cat_col in temp01.dtypes if cat_col[1] == 'string']
len(catcols)

16

In [13]:
# for _ in catcols:
#     agg_cols.remove(_)
# len(agg_cols)

In [11]:
for _ in catcols:
    agg_cols.remove(_)
len(agg_cols)

545

#### 4.1 Usando Apenas Flags Temporais

* Flag Temporal: U3M

In [12]:
new_cols = []

temp_flag = flags_temporais[0]

for agg_col in agg_cols:
  if 'DPD' in agg_col or 'DAY' in agg_col:
    new_cols.append(round(max(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"QT_MAX_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(min(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"QT_MIN_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    new_cols.append(round(sum(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_TOT_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(median(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_MED_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(max(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_MAX_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(min(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_MIN_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))


new_cols = tuple(new_cols)

temp03 = temp02.groupBy("SK_ID_CURR").agg(*new_cols).orderBy("SK_ID_CURR")

temp03.count()

338857

In [13]:
temp03 = temp03.repartition(1)
temp03.write.parquet("./BASES_FEAT_ENG/PREV_APP_1")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Spark\python\pyspark\errors\exceptions\captured.py", line 169, in deco
    return f(*a, **kw)
  File "C:\Spark\python\lib\py4j-0.10.9.7-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <exception str() failed>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Spark\python\lib\py4j-0.10.9.7-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\Leonardo\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Spark\python\lib\py4j-0.10.9.7-src.zip\py4j\ja

Py4JError: org does not exist in the JVM

In [ ]:
# Liberando Memória
temp03 = None
del(temp03)

* Flag Temporal: U6M

In [ ]:
new_cols = []

temp_flag = flags_temporais[1]

for agg_col in agg_cols:
  if 'DPD' in agg_col or 'DAY' in agg_col:
    new_cols.append(round(max(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"QT_MAX_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(min(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"QT_MIN_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
  else:
    new_cols.append(round(sum(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_TOT_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(median(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_MED_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(max(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_MAX_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))
    new_cols.append(round(min(when(col(temp_flag) == 1, col(agg_col))), 2).alias(f"VL_MIN_{agg_col.upper()}_{temp_flag.upper()}_PREVIOUS_APPLICATION"))


new_cols = tuple(new_cols)

temp04 = temp02.groupBy("SK_ID_CURR").agg(*new_cols).orderBy("SK_ID_CURR")

temp04.count()

In [ ]:
temp04 = temp04.repartition(1)
temp04.write.mode("overwrite").option("compression", "gzip").parquet("./BASES_FEAT_ENG/PREV_APP_2")